In [14]:
import pandas as pd
from pandas_ods_reader import read_ods
import json
import numpy as np

ods_path = "./Cities.ods"
data = read_ods(ods_path, 1, headers=True)
data.index = data.index + 1

json_path = './Nodes.json'
with open(json_path) as f:
    nodes = json.load(f)

#themes_idx = []
#themes = data['Тематика'].str.split(',', 1, expand = True)
#for theme in themes[0]:
#    themes_idx.append(nodes['Theme'].index(theme))
#themes[0] = themes_idx

del data['Город']
del data['Историческая личность']

themes = nodes['Тематика']
for theme in themes:
    data[theme] = data['Тематика'].map(lambda e: 1 if (theme in e) else 0)
del data['Тематика']

data['Вид промысла'], unique = pd.factorize(data['Вид промысла'])

'''
epochs = nodes['Эпоха']
for epoch in epochs:
    if data['Историческая эпоха'] != None:
        data[epoch] = data['Историческая эпоха'].map(lambda e: 1 if (theme in e) else 0)
del data['Историческая эпоха']

display(data)
'''

data['ЗК'], unique = pd.factorize(data['ЗК'])
data['ЗК'] += 1

data['Область'], unique = pd.factorize(data['Область'])
data['Направление'], unique = pd.factorize(data['Направление'])

data['Население'] = [elem.replace('тыс', '') for elem in data['Население']]
display(data)

,Вид промысла,Историческая эпоха,ЗК,Область,Направление,Расстояние от Москвы,Население,Гастрономия,История,Паломничество,Пляжный отдых,Промысел
1,-1,Личность,1,0,0,95.0,107,0,1,0,0,0
2,-1,None,1,0,0,200.0,"4,5",0,0,1,0,0
3,-1,Личность,0,1,1,120.0,"10,9",0,1,0,0,0
4,-1,"Столицы древней Руси, Из варяг в греки",0,2,2,580.0,"219,9",0,1,0,0,0
5,-1,Северо-Восточная Русь,1,0,0,190.0,"346,9",0,1,0,0,0
6,0,None,0,3,0,470.0,"305,4",0,0,0,0,1
7,-1,None,0,4,3,70.0,"35,8",0,0,1,0,0
8,1,None,0,4,4,45.0,"253,6",0,0,0,1,1
9,0,None,1,5,0,250.0,"408,9",0,0,0,0,1
10,2,None,0,6,5,1200.0,"631,2",0,0,0,0,1


In [15]:
# Евклидова мера близости

def Euclidean_measure(node1, node2):
    sumEvk = 0
    for elem in range(len(node1)):
        sumEvk += pow((node1[elem] - node2[elem]), 2)
    
    return pow(sumEvk, 1/2)

In [21]:
# Расстояние городских кварталов

def City_block_distance(node1, node2):
    sumCity = 0
    for elem in range(len(node1)):
        sumCity += abs(node1[elem] - node2[elem])
    
    return sumCity

In [23]:
# Косинусная мера близости

def Cosine_measure(node1, node2):
    sumMulty = 0
    for elem in range(len(node1)):
        sumMulty += node1[elem] * node2[elem]
    
    sum1 = 0
    for elem in range(len(node1)):
        sum1 += pow(node1[elem], 2)
    sum1 = pow(sum1, 1/2)
    
    sum2 = 0
    for elem in range(len(node2)):
        sum2 += pow(node2[elem], 2)
    sum2 = pow(sum2, 1/2)
    
    return (sumMulty / (sum1 * sum2))

In [28]:
# Расстояние Чебышева

def Chebyshev_measure(node1, node2):
    distance = []
    for elem in range(len(node1)):
        cur_dist = abs(node1[elem] - node2[elem])
        distance.append(cur_dist)
    
    return max(distance)

In [39]:
# Расстояние Минковского

def Minkowski_measure(p, node1, node2):
    sumMink = 0
    for elem in range(len(node1)):
        curDiff = node1[elem] - node2[elem]
        sumMink += np.sign(curDiff) * pow(abs(curDiff), p)
    
    return pow(sumMink, 1/p)

In [41]:
res1 = Euclidean_measure(data['Область'].values, data['Направление'].values)
res2 = City_block_distance(data['Область'].values, data['Направление'].values)
res3 = Cosine_measure(data['Область'].values, data['Направление'].values)
res4 = Chebyshev_distance(data['Область'].values, data['Направление'].values)
res5 = Minkowski_measure(0.25, data['Область'].values, data['Направление'].values)

display(res5)

460922.97987082275